In [3]:
import bs4 as bs
import numpy as np
import pandas as pd
import requests
import yfinance as yf
import datetime
import requests
import pickle
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [20]:
todayDt   = datetime.date.today()
startDt   = '2019-01-01'
endDt     = str(todayDt)[0:10]

In [26]:
with open("IndexComponents.pickle","rb") as f:
    [_SP500_df, _NQ100_df, _DJ30_df, rus_df] = pickle.load(f)


In [22]:
_SP500 = _SP500_df.index.tolist()
_NQ100 = _NQ100_df.index.tolist()
_DJ30 = _DJ30_df.index.tolist()

In [23]:
index_SP500  = yf.download(['^GSPC'], start=startDt, end=endDt)
index_NQ100  = yf.download(['^IXIC'], start=startDt, end=endDt)
index_DJ30   = yf.download(['^DJI'],  start=startDt, end=endDt)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [24]:
stocks_SP500 = yf.download([a for a in _SP500], start=startDt, end=endDt)

[*********************100%***********************]  505 of 505 completed

2 Failed downloads:
- BRK.B: No data found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted


In [25]:
stocks_NQ100 = yf.download([a for a in _NQ100], start=startDt, end=endDt)

[*********************100%***********************]  103 of 103 completed


In [2]:
stocks_DJ30  = yf.download([a for a in _DJ30],  start=startDt, end=endDt)

NameError: name '_DJ30' is not defined

In [27]:
statlib = {
    'SP500': {'Compo': stocks_SP500, 'Index': index_SP500, 'Name': 'S&P 500'},
    'NQ100': {'Compo': stocks_NQ100, 'Index': index_NQ100, 'Name': 'Nasdaq 100'},
    'DJ30':  {'Compo': stocks_DJ30,  'Index': index_DJ30,  'Name': 'DowJones 30'},
    }

In [29]:
def IndexOutperformers(refindex, period_ending, lookback_days, margin):
    if refindex in statlib.keys():
        compo_DF = statlib[refindex]['Compo']
        index_DF = statlib[refindex]['Index']
        _name    = statlib[refindex]['Name']
    
    if period_ending in index_DF.index:
        compo_slice  = compo_DF['Adj Close'].pct_change(lookback_days).loc[period_ending]
        index_return = index_DF['Adj Close'].pct_change(lookback_days).loc[period_ending]
        hurdle       = index_return + margin
        answer       = compo_slice[compo_slice>hurdle]
        print(_name)
        print("Date:", period_ending, "; Returns", lookback_days, "days prior")
        print("Index \t", round(index_return*100,2),"%","\n")
        print("Outperformers", "Total #", len(answer))
        for tick, value in answer.items():
            print(f"{tick} \t {round(value*100,2):5.2f} %")
        return

In [32]:
IndexOutperformers(refindex = 'DJ30', 
                   period_ending = '2020-06-12', 
                   lookback_days = 14, 
                   margin = 0.05)

DowJones 30
Date: 2020-06-12 ; Returns 14 days prior
Index 	 4.66 % 

Outperformers Total # 6
AXP 	 13.83 %
BA 	 37.80 %
DOW 	 16.19 %
GS 	 12.84 %
JPM 	 11.62 %
TRV 	 13.97 %


In [34]:
def IndexUnderperformers(refindex, period_ending, lookback_days, margin):
    if refindex in statlib.keys():
        compo_DF = statlib[refindex]['Compo']
        index_DF = statlib[refindex]['Index']
        _name    = statlib[refindex]['Name']

    if period_ending in index_DF.index:
        compo_slice  = compo_DF['Adj Close'].pct_change(lookback_days).loc[period_ending]
        index_return = index_DF['Adj Close'].pct_change(lookback_days).loc[period_ending]
        hurdle       = index_return - margin
        answer       = compo_slice[compo_slice<hurdle]
        
        print("Date:", period_ending, "; Returns", lookback_days, "days prior")
        print("Index \t", round(index_return*100,2),"%","\n")
        print("Underperformers", "Total #", len(answer) )
        for tick, value in answer.items():
            print(f"{tick} \t {round(value*100,2):5.2f} %")
        return

In [35]:
IndexUnderperformers(refindex = 'DJ30', 
                     period_ending = '2020-06-12', 
                     lookback_days = 14, 
                     margin = 0.01)

Date: 2020-06-12 ; Returns 14 days prior
Index 	 4.66 % 

Underperformers Total # 17
CSCO 	  0.38 %
CVX 	  2.34 %
DIS 	 -2.14 %
HD 	  0.83 %
IBM 	  2.97 %
INTC 	 -4.71 %
JNJ 	 -1.54 %
KO 	  2.19 %
MCD 	  3.27 %
MRK 	  0.70 %
MSFT 	  2.31 %
NKE 	  3.12 %
PFE 	 -10.00 %
PG 	  2.68 %
UNH 	 -1.65 %
V 	  0.73 %
WMT 	 -5.30 %


In [40]:
IndexOutperformers(refindex = 'SP500', 
                   period_ending = '2020-06-12', 
                   lookback_days = 4, 
                   margin = 0.0)

S&P 500
Date: 2020-06-12 ; Returns 4 days prior
Index 	 -5.91 % 

Outperformers Total # 141
A 	 -4.63 %
AAPL 	  1.60 %
ABBV 	 -2.25 %
ABMD 	 -4.59 %
ABT 	 -3.82 %
ACN 	 -4.59 %
ADBE 	  2.20 %
ADI 	 -4.28 %
ADSK 	 -4.52 %
AKAM 	 -2.94 %
AMAT 	 -5.59 %
AMD 	  1.00 %
AMGN 	 -3.48 %
AMT 	 -3.30 %
AMZN 	  0.83 %
ANET 	 -3.89 %
APD 	 -5.83 %
ARE 	 -0.51 %
ATO 	 -5.28 %
ATVI 	  1.34 %
AVGO 	 -5.44 %
AWK 	 -3.93 %
AZO 	 -3.78 %
BAX 	 -5.42 %
BLK 	 -5.43 %
BR 	 -4.17 %
CAG 	 -4.48 %
CBOE 	 -3.41 %
CCI 	 -4.23 %
CDNS 	 -2.90 %
CHD 	 -2.23 %
CHTR 	 -3.29 %
CL 	 -0.98 %
CLX 	  3.20 %
CMG 	 -5.47 %
CMS 	 -4.39 %
CNP 	 -5.71 %
COST 	 -2.76 %
CPB 	 -5.65 %
CPRT 	 -5.76 %
CRM 	 -0.82 %
CTXS 	  0.61 %
D 	 -3.76 %
DG 	 -1.08 %
DGX 	 -5.31 %
DHI 	 -2.03 %
DHR 	 -1.82 %
DLR 	 -3.14 %
DPZ 	  0.83 %
DRE 	 -5.00 %
DUK 	 -5.63 %
DXCM 	  0.44 %
EA 	  2.06 %
EBAY 	 -2.18 %
ED 	 -3.00 %
EQIX 	  0.06 %
ES 	 -3.93 %
EXPD 	 -5.58 %
FB 	 -1.22 %
FFIV 	 -4.07 %
FIS 	 -3.61 %
FTNT 	 -5.21 %
GILD 	 -4.05 %
GIS 	 -3.20 

In [41]:
IndexUnderperformers(refindex = 'SP500',
                     period_ending = '2020-06-12', 
                     lookback_days = 4, 
                     margin = 0.0)

Date: 2020-06-12 ; Returns 4 days prior
Index 	 -5.91 % 

Underperformers Total # 362
AAL 	 -17.58 %
AAP 	 -7.09 %
ABC 	 -7.26 %
ADM 	 -8.07 %
ADP 	 -8.24 %
ADS 	 -21.18 %
AEE 	 -6.48 %
AEP 	 -6.95 %
AES 	 -15.36 %
AFL 	 -12.97 %
AIG 	 -15.37 %
AIV 	 -10.02 %
AIZ 	 -8.84 %
AJG 	 -6.46 %
ALB 	 -10.45 %
ALGN 	 -13.24 %
ALK 	 -23.43 %
ALL 	 -10.55 %
ALLE 	 -11.36 %
ALXN 	 -8.72 %
AMCR 	 -6.57 %
AME 	 -8.46 %
AMP 	 -9.87 %
ANSS 	 -7.04 %
ANTM 	 -11.31 %
AON 	 -8.11 %
AOS 	 -7.72 %
APA 	 -22.34 %
APH 	 -8.71 %
APTV 	 -13.53 %
AVB 	 -8.04 %
AVY 	 -8.92 %
AXP 	 -10.55 %
BA 	 -17.78 %
BAC 	 -13.17 %
BBY 	 -6.44 %
BDX 	 -6.62 %
BEN 	 -10.07 %
BIIB 	 -7.30 %
BK 	 -9.10 %
BKNG 	 -11.79 %
BKR 	 -16.37 %
BLL 	 -7.76 %
BMY 	 -8.56 %
BSX 	 -9.61 %
BWA 	 -10.58 %
BXP 	 -9.91 %
C 	 -14.68 %
CAH 	 -10.82 %
CARR 	 -11.61 %
CAT 	 -10.58 %
CB 	 -10.67 %
CBRE 	 -14.27 %
CCL 	 -19.79 %
CDW 	 -11.06 %
CE 	 -10.34 %
CERN 	 -9.33 %
CF 	 -10.86 %
CFG 	 -15.09 %
CHRW 	 -8.66 %
CI 	 -11.96 %
CINF 	 -14.56 %
CMA 	 